In [2]:
import numpy as np
import pandas as pd
from numpy.linalg import norm, inv, eigh
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

data = pd.read_csv('/content/Real estate.csv', usecols=(2, 3, 4, 5, 6, 7))
xdata = data[['X2 house age', 'X3 distance to the nearest MRT station', 'X4 number of convenience stores', 'X5 latitude', 'X6 longitude']]
ydata = data['Y house price of unit area']

scaler = StandardScaler()
xdata_scaled = scaler.fit_transform(xdata)

X = np.hstack([np.ones((xdata_scaled.shape[0], 1)), xdata_scaled])
y = ydata.values

In [61]:
# Classical Newton's Method

def f(x, X, y):
  return 0.5 * np.sum((X @ x - y)**2)

def grad_f(x, X, y):
  return X.T @ (X @ x - y)

def hessian_f(X):
  return X.T @ X

def newtons_method(x0, epsilon, X, y):
  xk = x0
  k = 0
  max_iter = 1000
  while norm(grad_f(xk, X, y)) > epsilon and k < max_iter:
    Hk = hessian_f(X)
    gk = grad_f(xk, X, y)
    dk = -1 * inv(Hk) @ gk
    alpha_k = 1
    xk = xk + alpha_k * dk
    k += 1
  return xk

x0 = np.zeros(X.shape[1])
epsilon = 1e-6

x_star = newtons_method(x0, epsilon, X, y)

print("Optimal solution:", x_star)


Optimal solution: [37.98019324 -3.05992858 -5.36894108  3.42160882  2.94717174 -0.11964695]


In [62]:
#Modified newton

def f(x, X, y):
  return 0.5 * np.sum((X @ x - y)**2)

def grad_f(x, X, y):
  return X.T @ (X @ x - y)

def hessian_reg(Hk, delta):
  eigvals, _ = eigh(Hk)
  min_eigval = np.min(eigvals)
  if min_eigval >= delta:
    return Hk
  else:
    return Hk + (delta - min_eigval) * np.eye(Hk.shape[0])

def armijo_wolfe(xk, dk, X, y, fk, grad_fk, alpha_init=1.0, c1=1e-4, c2=0.9):
  alpha = alpha_init
  while alpha > 1e-10:
    x_next = xk + alpha * dk
    fk_next = f(x_next, X, y)
    phi1_alpha = fk + c1 * alpha * grad_fk @ dk

    if fk_next <= phi1_alpha:
      grad_fk_next = grad_f(x_next, X, y)
      phi1_prime_alpha = grad_fk.T @ dk
      phi1_prime_alpha_next = grad_fk_next @ dk

      if phi1_prime_alpha_next >= c2 * phi1_prime_alpha:
        return alpha

    alpha *= 0.5

  return alpha

def modified_newtons_method(x0, epsilon, delta, X, y):
    xk = x0
    k = 0
    max_iter = 1000
    while norm(grad_f(xk, X, y)) > epsilon and k < max_iter:
        Hk = X.T @ X
        gk = grad_f(xk, X, y)
        Hk_reg = hessian_reg(Hk, delta)
        dk = -1 * inv(Hk_reg) @ gk
        alpha_k = armijo_wolfe(xk, dk, X, y, f(xk, X, y), gk)
        xk = xk + alpha_k * dk
        k += 1

    return xk

x0 = np.zeros(X.shape[1])
epsilon = 1e-6
delta = 1e-3
x_star = modified_newtons_method(x0, epsilon, delta, X, y)

print("Optimal solution:", x_star)

Optimal solution: [37.98019324 -3.05992858 -5.36894108  3.42160882  2.94717174 -0.11964695]


In [63]:
#Modified Using Trust region

def f(x, X, y):
  return 0.5 * np.sum((X @ x - y)**2)

def grad_f(x, X, y):
  return X.T @ (X @ x - y)

def hessian_f(X):
  return X.T @ X

def trust_region_newton(x0, epsilon, Delta_0, X, y):
  xk = x0
  Delta_k = Delta_0
  k = 0
  max_iter = 1000

  while norm(grad_f(xk, X, y)) > epsilon and k < max_iter:
    Omega_k = np.linalg.norm(xk - X) <= Delta_k
    pk = -inv(hessian_f(X)) @ grad_f(xk, X, y)
    x_next = xk + pk
    actual_reduction = f(xk, X, y) - f(x_next, X, y)
    predicted_reduction = -grad_f(xk, X, y).T @ pk - 0.5 * pk.T @ hessian_f(X) @ pk
    Rk = actual_reduction / predicted_reduction
    if Rk < 0.25:
      Delta_k = 0.5 * Delta_k
    elif Rk > 0.75 and np.linalg.norm(x_next - xk) == Delta_k:
      Delta_k = min(2 * Delta_k, Delta_0)
    else:
      Delta_k = Delta_k
    xk = x_next
    k += 1

  return xk

x0 = np.zeros(X.shape[1])
epsilon = 1e-6
Delta_0 = 0.1

x_star = trust_region_newton(x0, epsilon, Delta_0, X, y)
print("Optimal solution:", x_star)

Optimal solution: [37.98019324 -3.05992858 -5.36894108  3.42160882  2.94717174 -0.11964695]


In [64]:
#SD

def f(x, X, y):
  return 0.5 * np.sum((X @ x - y)**2)

def grad_f(x, X, y):
  return X.T @ (X @ x - y)

def armijo_wolfe(xk, dk, X, y, fk, grad_fk, alpha_init=1.0, c1=1e-4, c2=0.9):
  alpha = alpha_init
  while alpha > 1e-10:
    x_next = xk + alpha * dk
    fk_next = f(x_next, X, y)
    phi1_alpha = fk + c1 * alpha * grad_fk @ dk

    if fk_next <= phi1_alpha:
      grad_fk_next = grad_f(x_next, X, y)
      phi1_prime_alpha = grad_fk.T @ dk
      phi1_prime_alpha_next = grad_fk_next.T @ dk

      if phi1_prime_alpha_next >= c2 * phi1_prime_alpha:
        return alpha

    alpha *= 0.5

  return alpha

def steepest_descent(x0, epsilon, X, y):
  xk = x0
  k = 0
  max_iter = 100

  while norm(grad_f(xk, X, y)) > epsilon and k < max_iter:
    gk = grad_f(xk, X, y)
    dk = -gk
    alpha_k = armijo_wolfe(xk, dk, X, y, f(xk, X, y), gk)
    xk = xk + alpha_k * dk
    k += 1

  return xk

x0 = np.zeros(X.shape[1])
epsilon = 1e-6

x_star = steepest_descent(x0, epsilon, X, y)
print("Optimal solution:", x_star)

Optimal solution: [37.98019324 -3.05992858 -5.36894107  3.42160882  2.94717173 -0.11964695]


In [10]:
#quasi

def f(x, X, y):
  return 0.5 * np.sum((X @ x - y)**2)

def grad_f(x, X, y):
  return X.T @ (X @ x - y)

def armijo_wolfe(xk, dk, X, y, fk, grad_fk, alpha_init=1.0, c1=1e-4, c2=0.9):
  alpha = alpha_init
  while alpha > 1e-10:
    x_next = xk + alpha * dk
    fk_next = f(x_next, X, y)
    phi1_alpha = fk + c1 * alpha * grad_fk @ dk

    if fk_next <= phi1_alpha:
      grad_fk_next = grad_f(x_next, X, y)
      phi1_prime_alpha = grad_fk.T @ dk
      phi1_prime_alpha_next = grad_fk_next.T @ dk

      if phi1_prime_alpha_next >= c2 * phi1_prime_alpha:
        return alpha

    alpha *= 0.5

  return alpha

def quasi_newton_sr1(x0, epsilon, X, y):
  xk = x0
  k = 0
  max_iter = 1000
  n = len(x0)
  Bk = np.eye(n)
  while norm(grad_f(xk, X, y)) > epsilon and k < max_iter:
    gk = grad_f(xk, X, y)
    dk = -Bk @ gk
    alpha_k = armijo_wolfe(xk, dk, X, y, f(xk, X, y), gk)
    x_next = xk + alpha_k * dk
    delta_k = x_next - xk
    xk = x_next
    gk_next = grad_f(xk, X, y)
    gamma_k = gk_next - gk

    BG = Bk @ gamma_k
    numer = (delta_k - BG) @ ((delta_k - BG).T)
    denom = ((delta_k - BG).T) @ gamma_k
    frac = numer / denom
    B_next = Bk + frac
    k += 1

  return xk

x0 = np.zeros(X.shape[1])
epsilon = 1e-6
x_star = quasi_newton_sr1(x0, epsilon, X, y)

print("Optimal solution:", x_star)

Optimal solution: [37.98019324 -3.05992858 -5.36894107  3.42160882  2.94717173 -0.11964695]
